## Your first attempt at coding something

In [1]:
import json, requests

with open('my_file.json') as file:
    data = json.load(file)

# print(data)

# response = requests.get('https://httpbin.org/headers')
# print(response.json())

response = requests.post('https://httpbin.org/anything', json={'my_data': 'my_data'})
print(response.json())

with open('my_file_2.json', 'w') as file:
    json.dump(response.json(), file)

{'args': {}, 'data': '{"my_data": "my_data"}', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '22', 'Content-Type': 'application/json', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.25.1', 'X-Amzn-Trace-Id': 'Root=1-619d5d37-22d31cf324ecfe221c988a61'}, 'json': {'my_data': 'my_data'}, 'method': 'POST', 'origin': '71.14.235.146', 'url': 'https://httpbin.org/anything'}


## Writing methods, cleaning up style, using a helper library

In [2]:
import json
import pathlib

import requests


def read_json(path):
    content = pathlib.Path(path).read_text()
    return json.loads(content)

def write_json(obj, path):
    content = json.dumps(obj)
    pathlib.Path(path).write_text(content)

def get_headers():
    response = requests.get('https://httpbin.org/headers')
    return response.json()

def post_json(obj):
    response = requests.post('https://httpbin.org/anything', json=obj)
    return response.json()

def main():
    data = read_json('my_file.json')
    print(data)

    response = get_headers()
    print(response)

    data.update({'my_data': 'my_data'})
    response = post_json(data)
    print(response)

    write_json(response, 'my_file_2.json')


main()

{'x': 'y'}
{'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.25.1', 'X-Amzn-Trace-Id': 'Root=1-619d5d37-734ad083192af8ea45317fe3'}}
{'args': {}, 'data': '{"x": "y", "my_data": "my_data"}', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '32', 'Content-Type': 'application/json', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.25.1', 'X-Amzn-Trace-Id': 'Root=1-619d5d37-042f3171368fd67b1f39eba3'}, 'json': {'my_data': 'my_data', 'x': 'y'}, 'method': 'POST', 'origin': '71.14.235.146', 'url': 'https://httpbin.org/anything'}


## What if our data comes from a URL and not a file?  
## What if we want to send our data somewhere other than a file?  

In [3]:
import json
import pathlib
from pprint import pprint

import requests


def read_json(path):
    content = pathlib.Path(path).read_text()
    return json.loads(content)

def write_json(obj, path):
    content = json.dumps(obj)
    pathlib.Path(path).write_text(content)

def get_headers():
    response = requests.get('https://httpbin.org/headers')
    return response.json()

def post_json(obj, url='https://httpbin.org/anything'):
    response = requests.post(url, json=obj)
    return response.json()

def read_data(data_source):
    if 'http' in data_source:
        response = requests.get(data_source)
        return response.json()
    else:
        return read_json(data_source)

def write_data(obj, path):
    if 'http' in path:
        post_json(obj, path)
    else:
        write_json(obj, path)
    
def main(data_source='my_file.json', output='my_file_2.json'):
    data = read_data(data_source)
    print(f'Data: {data}')

    response = get_headers()
    print('Headers:')
    pprint(response['headers'])

    data.update({'my_data': 'my_data'})
    response = post_json(data)
    print('Response:')
    pprint(response)

    write_data(response, output)


main('my_file.json', 'my_file_2.json')
print('='*120)
main('https://httpbin.org/uuid', 'https://httpbin.org/anything')

Data: {'x': 'y'}
Headers:
{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Host': 'httpbin.org',
 'User-Agent': 'python-requests/2.25.1',
 'X-Amzn-Trace-Id': 'Root=1-619d5d37-722b306d1a5cbde524a78997'}
Response:
{'args': {},
 'data': '{"x": "y", "my_data": "my_data"}',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
             'Accept-Encoding': 'gzip, deflate',
             'Content-Length': '32',
             'Content-Type': 'application/json',
             'Host': 'httpbin.org',
             'User-Agent': 'python-requests/2.25.1',
             'X-Amzn-Trace-Id': 'Root=1-619d5d37-4039a1205be8dfe96cd78358'},
 'json': {'my_data': 'my_data', 'x': 'y'},
 'method': 'POST',
 'origin': '71.14.235.146',
 'url': 'https://httpbin.org/anything'}
Data: {'uuid': '5b1ad3e5-ee48-45f5-91f4-9a8f409c9cac'}
Headers:
{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Host': 'httpbin.org',
 'User-Agent': 'python-requests/2.25.1',
 'X-Amzn-Trace-Id': 'Root=1-619d5d38-69352b373842a

## What if someone else wants to use our code?  
## Let's add some type hints and some options on what we print by switching to the logging library and assigning levels to messages.

In [4]:
import json
import pathlib
import logging
from typing import Union, Dict, List

import requests


def read_json(path: str) -> Union[Dict, List]:
    logging.info(f'Reading JSON from disk at {path}')
    content = pathlib.Path(path).read_text()
    return json.loads(content)

def write_json(obj: Union[Dict, List], path: str):
    logging.info(f'Writing JSON to disk at {path}')
    content = json.dumps(obj)
    pathlib.Path(path).write_text(content)

def get_headers() -> Dict:
    logging.info('Getting headers from httpbin.org')
    response = requests.get('https://httpbin.org/headers')
    return response.json()

def post_json(obj: Union[Dict, List], url: str = 'https://httpbin.org/anything') -> Union[Dict, List]:
    logging.info('Posting data to httbin.org')
    response = requests.post(url, json=obj)
    return response.json()

def read_data(data_source: str) -> Union[Dict, List]:
    logging.info(f'Reading from "{data_source}"')
    if 'http' in data_source:
        response = requests.get(data_source)
        return response.json()
    else:
        return read_json(data_source)

def write_data(obj: Union[Dict, List], path: str):
    logging.info(f'Writing to "{path}"')
    if 'http' in path:
        post_json(obj, path)
    else:
        write_json(obj, path)
    
def main(data_source: str = 'my_file.json', output: str = 'my_file_2.json'):
    data = read_data(data_source)
    logging.debug(f'Data: {data}')

    response = get_headers()
    logging.debug('Headers:')
    logging.debug(response['headers'])

    data.update({'my_data': 'my_data'})
    response = post_json(data)
    logging.debug('Response:')
    logging.debug(response)

    write_data(response, output)

In [5]:
# init root logger for session
logging.basicConfig(format='%(asctime)s [%(levelname)-7s] %(name)s:%(lineno)d | %(message)s')

In [6]:
logging.getLogger().setLevel(logging.INFO)

main('my_file.json', 'my_file_2.json')

2021-11-23 15:29:28,729 [INFO   ] root:30 | Reading from "my_file.json"
2021-11-23 15:29:28,730 [INFO   ] root:10 | Reading JSON from disk at my_file.json
2021-11-23 15:29:28,731 [INFO   ] root:20 | Getting headers from httpbin.org
2021-11-23 15:29:28,871 [INFO   ] root:25 | Posting data to httbin.org
2021-11-23 15:29:29,001 [INFO   ] root:38 | Writing to "my_file_2.json"
2021-11-23 15:29:29,002 [INFO   ] root:15 | Writing JSON to disk at my_file_2.json


In [7]:
logging.getLogger('root').setLevel(logging.DEBUG)

main('https://httpbin.org/uuid', 'https://httpbin.org/anything')

2021-11-23 15:29:29,008 [INFO   ] root:30 | Reading from "https://httpbin.org/uuid"
2021-11-23 15:29:29,011 [DEBUG  ] urllib3.connectionpool:971 | Starting new HTTPS connection (1): httpbin.org:443
2021-11-23 15:29:29,146 [DEBUG  ] urllib3.connectionpool:452 | https://httpbin.org:443 "GET /uuid HTTP/1.1" 200 53
2021-11-23 15:29:29,150 [DEBUG  ] root:46 | Data: {'uuid': '43076a83-89dc-4485-a95c-099e7420590c'}
2021-11-23 15:29:29,151 [INFO   ] root:20 | Getting headers from httpbin.org
2021-11-23 15:29:29,154 [DEBUG  ] urllib3.connectionpool:971 | Starting new HTTPS connection (1): httpbin.org:443
2021-11-23 15:29:29,285 [DEBUG  ] urllib3.connectionpool:452 | https://httpbin.org:443 "GET /headers HTTP/1.1" 200 225
2021-11-23 15:29:29,289 [DEBUG  ] root:49 | Headers:
2021-11-23 15:29:29,290 [DEBUG  ] root:50 | {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.25.1', 'X-Amzn-Trace-Id': 'Root=1-619d5d39-5911b4cd1f8c73a50ba082dd'}
2

## Helpful but now we're seeing urllib3 debug logs too. Let's make our own logger

In [8]:
import json
import pathlib
import logging
from typing import Union, Dict, List

import requests


logger = logging.getLogger('my_module')

def read_json(path: str) -> Union[Dict, List]:
    logger.info(f'Reading JSON from disk at {path}')
    content = pathlib.Path(path).read_text()
    return json.loads(content)

def write_json(obj: Union[Dict, List], path: str):
    logger.info(f'Writing JSON to disk at {path}')
    content = json.dumps(obj)
    pathlib.Path(path).write_text(content)

def get_headers() -> Dict:
    logger.info('Getting headers from httpbin.org')
    response = requests.get('https://httpbin.org/headers')
    return response.json()

def post_json(obj: Union[Dict, List], url: str = 'https://httpbin.org/anything') -> Union[Dict, List]:
    logger.info('Posting data to httbin.org')
    response = requests.post(url, json=obj)
    return response.json()

def read_data(data_source: str) -> Union[Dict, List]:
    logger.info(f'Reading from "{data_source}"')
    if 'http' in data_source:
        response = requests.get(data_source)
        return response.json()
    else:
        return read_json(data_source)

def write_data(obj: Union[Dict, List], path: str):
    logger.info(f'Writing to "{path}"')
    if 'http' in path:
        post_json(obj, path)
    else:
        write_json(obj, path)
    
def main(data_source: str = 'my_file.json', output: str = 'my_file_2.json'):
    data = read_data(data_source)
    logger.debug(f'Data: {data}')

    response = get_headers()
    logger.debug('Headers:')
    logger.debug(response['headers'])

    data.update({'my_data': 'my_data'})
    response = post_json(data)
    logger.debug('Response:')
    logger.debug(response)

    write_data(response, output)

In [9]:
logging.getLogger().setLevel(logging.INFO)
logging.getLogger('my_module').setLevel(logging.DEBUG)

main('https://httpbin.org/uuid', 'https://httpbin.org/anything')

2021-11-23 15:29:29,614 [INFO   ] my_module:32 | Reading from "https://httpbin.org/uuid"
2021-11-23 15:29:29,759 [DEBUG  ] my_module:48 | Data: {'uuid': '2ab781b9-df6b-4d1f-ac18-939e4aa73d9e'}
2021-11-23 15:29:29,760 [INFO   ] my_module:22 | Getting headers from httpbin.org
2021-11-23 15:29:29,902 [DEBUG  ] my_module:51 | Headers:
2021-11-23 15:29:29,903 [DEBUG  ] my_module:52 | {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.25.1', 'X-Amzn-Trace-Id': 'Root=1-619d5d39-1b46bee65fcce6e452064dd4'}
2021-11-23 15:29:29,903 [INFO   ] my_module:27 | Posting data to httbin.org
2021-11-23 15:29:30,041 [DEBUG  ] my_module:56 | Response:
2021-11-23 15:29:30,042 [DEBUG  ] my_module:57 | {'args': {}, 'data': '{"uuid": "2ab781b9-df6b-4d1f-ac18-939e4aa73d9e", "my_data": "my_data"}', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '70', 'Content-Type': 'application/json', 'Host': '